In [ ]:
!pip install -q bitsandbytes accelerator transformers langchain ctransformers aiofiles PyPDF2 faiss-gpu pypdf
!pip install -q ctransformers[cuda]>=0.2.24
!pip install -q pypdf sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.llms import CTransformers
from langchain.chains import QAGenerationChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
import os
import json
import time
# import uvicorn
import aiofiles
from PyPDF2 import PdfReader
import csv

In [ ]:
def load_llm():
    config = {'max_new_tokens': 10000, 'temperature': 0.1,"gpu_layers":100,"context_length":10000}
    llm = CTransformers(
        model = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
        model_file='mistral-7b-instruct-v0.1.Q4_K_S.gguf',
        model_type="mistral",
        config=config
    )
    return llm

In [ ]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader,TextLoader

def file_processing(file_path):

    # Load data from PDF
    loader=DirectoryLoader("/content/data",glob="*.txt",loader_cls=TextLoader)
    data=loader.load()

    question_gen = ''

    for page in data:
        question_gen += page.page_content

    splitter_ques_gen = RecursiveCharacterTextSplitter(
        chunk_size = 1200,
        chunk_overlap = 100
    )

    chunks_ques_gen = splitter_ques_gen.split_text(question_gen)

    document_ques_gen = [Document(page_content=t) for t in chunks_ques_gen]

    splitter_ans_gen = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 50
    )


    document_answer_gen = splitter_ans_gen.split_documents(
        document_ques_gen
    )

    return document_ques_gen, document_answer_gen

In [ ]:
from langchain.vectorstores import FAISS
import re
def llm_pipeline(file_path):
    document_ques_gen, document_answer_gen = file_processing(file_path)
    llm_ques_gen_pipeline = load_llm()

    # prompt_template = """
    # You are an expert at creating questions based on materials and documentation.
    # Your goal is to prepare a set of questions.
    # You do this by asking questions about the text below:

    # ------------
    # {text}
    # ------------

    # Create questions that will prepare the end users.
    # Make sure not to lose any important information.

    # QUESTIONS:
    # """
#     prompt_template = """
# Based on the text below, please generate a comprehensive set of 30 questions that test understanding of the key details and ideas presented. Make sure to cover all aspects of the content in an unbiased way.
# Text:
# ------------
# {text}
# ------------
# Questions:
# """
    prompt_template = """
    Based on the text below, please generate a comprehensive set of 30 questions that test understanding of the key details and ideas presented, including information related to any URLs mentioned. Make sure to cover all aspects of the content in an unbiased way.
    Text:
    ------------
    {text}
    ------------
    Questions:
    """

    PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["text"])
    refine_template = ("""
    You are an expert at creating questions based on material and documentation.
    We have received some practice questions to a certain extent: {existing_answer}.
    Based on the text below, please generate a comprehensive set of questions that test understanding of the key details and ideas presented, including information related to any URLs mentioned. Make sure to cover all aspects of the content in an unbiased way.
    Text:
    ------------
    {text}
    ------------
    QUESTIONS:
    """
    )
    # refine_template = ("""
    # You are an expert at creating 30 questions based on material and documentation.
    # We have received some practice questions to a certain extent: {existing_answer}.
    # We have the option to refine the existing questions or add new ones.
    # (only if necessary) with some more context below.
    # ------------
    # {text}
    # ------------
    # Given the new context, refine the original questions in English.
    # If the context is not helpful, please provide the original questions.
    # QUESTIONS:
    # """
    # )

    REFINE_PROMPT_QUESTIONS = PromptTemplate(
        input_variables=["existing_answer", "text"],
        template=refine_template,
    )

    ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline,
                                            chain_type = "refine",
                                            verbose = True,
                                            question_prompt=PROMPT_QUESTIONS,
                                            refine_prompt=REFINE_PROMPT_QUESTIONS)

    ques = ques_gen_chain.run(document_ques_gen)

    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

    vector_store = FAISS.from_documents(document_answer_gen, embeddings)

    llm_answer_gen = load_llm()

    ques_list = ques.split("\n")
    filtered_ques_list = [re.sub(r'^\d+\.\s*', '', element) for element in ques_list if element.endswith('?') or element.endswith('.')]
    answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen,
                                                chain_type="stuff",
                                                retriever=vector_store.as_retriever())

    return answer_generation_chain, filtered_ques_list


In [ ]:
from langchain.vectorstores import FAISS
import re
from langchain.chains import LLMChain

def llm_pipeline(file_path):
    document_ques_gen, document_answer_gen = file_processing(file_path)
    llm_ques_gen_pipeline = load_llm()

    # prompt_template = """
    # You are an expert at creating questions based on materials and documentation.
    # Your goal is to prepare a set of questions.
    # You do this by asking questions about the text below:

    # ------------
    # {text}
    # ------------

    # Create questions that will prepare the end users.
    # Make sure not to lose any important information.

    # QUESTIONS:
    # """
#     prompt_template = """
# Based on the text below, please generate a comprehensive set of 30 questions that test understanding of the key details and ideas presented. Make sure to cover all aspects of the content in an unbiased way.
# Text:
# ------------
# {text}
# ------------
# Questions:
# """
    prompt_template = """
    Based on the text below, please generate a comprehensive set of 30 questions that test understanding of the key details and ideas presented, including information related to any URLs mentioned. Make sure to cover all aspects of the content in an unbiased way.
    Text:
    ------------
    {text}
    ------------
    Questions:
    """

    PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["text"])

    ques_gen_chain=LLMChain(llm=llm_ques_gen_pipeline, prompt=PROMPT_QUESTIONS,verbose=True)
    ques=""
    for doc in document_ques_gen:
      ques += ques_gen_chain.run(doc)

    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

    vector_store = FAISS.from_documents(document_answer_gen, embeddings)

    llm_answer_gen = load_llm()

    ques_list = ques.split("\n")
    filtered_ques_list = [re.sub(r'^\d+\.\s*', '', element) for element in ques_list if element.endswith('?') or element.endswith('.')]
    answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen,
                                                chain_type="stuff",
                                                retriever=vector_store.as_retriever())

    return answer_generation_chain, filtered_ques_list


In [ ]:
answer_generation_chain, ques_list = llm_pipeline("/content/data")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:

    Based on the text below, please generate a comprehensive set of 30 questions that test understanding of the key details and ideas presented, including information related to any URLs mentioned. Make sure to cover all aspects of the content in an unbiased way.  
    Text:  
    ------------
    page_content='Page No.1: ABOUT AHS\n\nABOUT AHS\n\nWHO WE ARE\n\nAlberta Health Services (AHS) is proud to be part of Canada’s first and largest provincewide, integrated health system, responsible for delivering health services to more than 4.4 million people living in Alberta, as well as to some residents of Saskatchewan, B.C. and the Northwest Territories.\n\nAHS has more than 112,300 direct AHS employees (excluding Covenant Health and other contracted service providers) and more than 13,000 staff working in AHS’ wholly-owned subsidiaries such as Alberta Precision Laboratories, Carewest and CapitalCare Group. AHS is also supported

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
len(ques_list)

142

In [ ]:
ques_list=['What is the name of the organization?',
 'What is the purpose of the organization?',
 'Who does the organization serve?',
 'How many people does the organization serve?',
 'Which provinces does the organization serve?',
 'Which territories does the organization serve?',
 'How many direct AHS employees does the organization have (excluding Covenant Health and other contracted service providers)?',
 'How many staff members work in AHS’ wholly-owned subsidiaries?',
 'What are the names of AHS’ wholly-owned subsidiaries?',
 'How many volunteers does the organization have?',
 'How many physicians practice in Alberta?',
 'How many of these physicians are members of the AHS medical staff?',
 'What types of appointments do these physicians hold (active, temporary or community)?',
 'What is the name of the province where the organization is located?',
 'What is the name of the largest province in Canada?',
 'What is the name of the first and largest provincewide, integrated health system in Canada?',
 'How many people does this integrated health system serve?',
 'Which provinces does this integrated health system serve?',
 'Which territories does this integrated health system serve?',
 'What is the name of the organization that provides contracted service providers to AHS?',
 'What is the name of the organization that supports AHS with approximately 9,100 volunteers?',
 'What is the name of the organization that supports AHS with more than 11,000 physicians practicing in Alberta?',
 'How many direct AHS employees does this integrated health system have (excluding Covenant Health and other contracted service providers)?',
 'How many staff members work in this integrated health system’s wholly-owned subsidiaries?',
 'What are the names of these wholly-owned subsidiaries?',
 'How many volunteers does this integrated health system have?',
 'How many physicians practice in Alberta and are members of this integrated health system’s medical staff?',
 'What types of appointments do these physicians hold (active, temporary or community)?',
 'What is the name of the province where this integrated health system is located?',
 'What is the name of the largest province in Canada where this integrated health system serves people?']

In [ ]:
import re
filtered_sentences = [re.sub(r'^\s*\d+\.\s*', '', sentence) for sentence in ques_list]


filtered_sentences

['What is the name of the organization?',
 'What is the purpose of the organization?',
 'Who does the organization serve?',
 'How many people does the organization serve?',
 'Which provinces does the organization serve?',
 'Which territories does the organization serve?',
 'How many direct AHS employees does the organization have (excluding Covenant Health and other contracted service providers)?',
 'How many staff members work in AHS’ wholly-owned subsidiaries?',
 'What are the names of AHS’ wholly-owned subsidiaries?',
 'How many volunteers does the organization have?',
 'How many physicians practice in Alberta?',
 'How many of these physicians are members of the AHS medical staff?',
 'What types of appointments do these physicians hold (active, temporary or community)?',
 'What is the name of the province where the organization is located?',
 'What is the name of the largest province in Canada?',
 'What is the name of the first and largest provincewide, integrated health system in C

In [ ]:
ques_list=['What is the name of the organization that delivers health services to more than 4.4 million people living in Alberta?',
 'What is the purpose of Alberta Health Services (AHS)?',
 'How many direct AHS employees are there, excluding Covenant Health and other contracted service providers?',
 'How many staff work in AHS’ wholly-owned subsidiaries such as Alberta Precision Laboratories, Carewest and CapitalCare Group?',
 'What is the number of volunteers that support AHS?',
 'How many physicians practice in Alberta, approximately how many of them are members of the AHS medical staff?',
 'Where do students from Alberta’s universities and colleges receive clinical education?',
 'Where do students from universities and colleges outside of Alberta receive clinical education?',
 'What is the purpose of clinical education in AHS facilities and community locations?',
 'How many people does AHS serve in total?',
 'Which provinces does AHS serve, besides Alberta?',
 'What are the main responsibilities of AHS?',
 'What is the role of Covenant Health and other contracted service providers in relation to AHS?',
 'How many people does AHS serve in Saskatchewan, B.C. and the Northwest Territories?',
 'What is the purpose of excluding Covenant Health and other contracted service providers from the direct AHS employee count?',
 'What are the main goals of AHS?',
 'How does AHS work to achieve its goals?',
 'What is the role of volunteers in achieving AHS’ goals?',
 'What is the role of physicians in achieving AHS’ goals?',
 'What is the role of students from universities and colleges in achieving AHS’ goals?',
 'How does AHS work to ensure that its services are accessible to all residents of Alberta, regardless of their location or socioeconomic status?',
 'What is the role of community locations in providing health services to residents of Alberta?',
 'What is the role of AHS’ wholly-owned subsidiaries in providing health services to residents of Alberta?',
 'How does AHS work to ensure that its services are integrated and coordinated with other healthcare providers in Alberta?',
 'What is the role of AHS in ensuring that patients receive high-quality care?',
 'What is the role of AHS in ensuring that patients have access to the latest medical technologies and treatments?',
 'How does AHS work to ensure that its services are safe and effective?',
 'What is the role of AHS in ensuring that patients receive appropriate care for their specific needs?',
 'What is the role of AHS in ensuring that patients have access to healthcare services that meet their cultural and linguistic needs?',
 'What is the purpose of an equity partnership in primary care networks for AHS?1. What is the name of the organization mentioned in the text?',
 'What is the purpose of AHS?',
 'What are the three geographically based health authorities that were brought together by AHS?',
 'When was ground ambulance service added to the responsibilities of AHS?',
 'When were the services moved from municipalities to AHS?',
 'What is the vision of AHS?',
 'What is the shared purpose of AHS?',
 'What are the common goals of AHS?',
 'What is the People Strategy of AHS?',
 'What is the link to the People Strategy of AHS?',
 'What are the Employer Awards mentioned in the text?',
 'What is Canada’s Best Diversity Employers?',
 'What is Alberta’s Top 75 Employers?',
 'What is the link to the Employer Awards of AHS?',
 'What is the infographic mentioned in the text?',
 'What is the purpose of the infographic?',
 'What are the benefits of working at AHS according to the infographic?',
 'What is the link to the infographic of AHS?',
 'What is the mission statement of AHS?',
 'What is the role of leadership in achieving the vision and purpose of AHS?',
 'What are the resources needed by AHS to work together effectively?',
 'What is the link to the Leadership Strategy of AHS?',
 'What are the benefits of working at AHS according to the Leadership Strategy?',
 'What is the role of leadership in developing excellent leadership at AHS?',
 'What are the values that AHS respects, values and supports its people?',
 'What is the link to the Values Strategy of AHS?',
 'What are the benefits of working at AHS according to the Values Strategy?',
 'What is the role of leadership in creating a culture in which all feel safe, healthy and valued?',
 'What are the benefits of working at AHS according to the Culture Strategy?',
 'What is the link to the Culture Strategy of AHS?1. What is the name of the organization?',
 'What is the purpose of the infographic?',
 'Who is Dr. John Cowell and what is his role in AHS?',
 'Where can I find the Health Care Action Plan report?',
 'What are the bylaws and policies of AHS?',
 'What is the vision, mission, and strategic direction of AHS?',
 'What publications does AHS have available?',
 'What is the health plan and business plan of AHS?',
 'Where can I find contact information for AHS?',
 'How many pages are in the Health Care Action Plan report?',
 'Who wrote the Health Care Action Plan report?',
 'When was the Health Care Action Plan report released?',
 "What is the purpose of the Bylaws & Policies page on AHS's website?",
 'What are some examples of the policies listed on the Bylaws & Policies page?',
 'How can I find out more information about the Vision, Mission & Strategic Direction of AHS?',
 "What is the purpose of the Publications & Transparency page on AHS's website?",
 'What types of publications are listed on the Publications & Transparency page?',
 'How can I find out more information about the Health Plan & Business Plan of AHS?',
 "What is the purpose of the Contact Us page on AHS's website?",
 'What information is included on the Contact Us page?',
 'What is the purpose of the infographic?',
 'How many sections are in the infographic?',
 'What information is included in each section of the infographic?',
 'Who created the infographic?',
 'When was the infographic created?',
 'What is the purpose of the Health Care Action Plan report?',
 'What are some key points from the Health Care Action Plan report?',
 'How does AHS plan to address the challenges outlined in the Health Care Action Plan report?',
 'What is the role of Dr. John Cowell in addressing these challenges?',
 'What information is included in this subheading?1. What is the purpose of Alberta Health Services valuing patient feedback?',
 'How will patient feedback help improve the quality of healthcare in Alberta?',
 'What are the different ways patients can provide feedback to Alberta Health Services?',
 'What is the electronic feedback form for patient feedback?',
 'What is the mail or phone feedback option for patient feedback?',
 'What is the purpose of the Alberta Health Care Insurance Plan?',
 'What is the Alberta Health Care Card?',
 'Where can patients find information about the Alberta Health Care Insurance Plan and Alberta Health Care Card?',
 'What are the emergency phone numbers for patients in Alberta?',
 'Where can patients find information about health advisory councils in Alberta?',
 'How can patients send a complaint to Environmental Public Health in Alberta?',
 'What is the Health Inspector in Alberta?',
 'Where can patients find information about parking at healthcare facilities in Alberta?',
 'What is Patient Concerns & Feedback in Alberta Health Services?',
 'How can patients provide feedback about services they have received from Alberta Health Services?',
 'What is the purpose of Patient Well Wishes in Alberta Health Services?',
 'How can patients email a hospitalized patient in Alberta?',
 'Where can patients find information about patient well wishes in Alberta Health Services?',
 'What are the different types of feedback that patients can provide to Alberta Health Services?',
 'How does Alberta Health Services use patient feedback to improve healthcare services?',
 'What is the role of patient feedback in shaping healthcare policies in Alberta?',
 'How does Alberta Health Services ensure patient privacy and confidentiality when collecting feedback?',
 'What are the benefits of providing feedback to Alberta Health Services?',
 'How can patients ensure that their feedback is taken seriously by Alberta Health Services?',
 'What are the potential consequences of not providing feedback to Alberta Health Services?',
 'How does Alberta Health Services measure the success of its patient feedback initiatives?',
 'What are some examples of successful patient feedback initiatives in Alberta Health Services?',
 'How can patients stay informed about patient feedback initiatives in Alberta Health Services?',
 'What is the role of healthcare providers in collecting and using patient feedback in Alberta?',
 "Where can someone find general inquiries related to children's health services?1. What is the name of the location mentioned in the text?",
 'Which floor of the North Tower is mentioned in the text?',
 'What is the address of the location mentioned in the text?',
 'What is the city mentioned in the text?',
 'What is the province mentioned in the text?',
 'What is the postal code mentioned in the text?',
 'What is the phone number mentioned in the text?',
 'What is the URL of the page that mentions the location?',
 'What is the URL of the page where you can report a problem?',
 'What is the URL of the page that goes to the top of the website?',
 'What is the name of the organization mentioned in the text?',
 'What is the purpose of the website feedback form mentioned in the text?',
 'What is the URL of the page where you can find more information about the organization?',
 'What is the URL of the page that provides information about the services offered by the organization?',
 'What is the URL of the page that provides information about the programs and initiatives offered by the organization?',
 'What is the URL of the page that provides information about the history of the organization?',
 'What is the URL of the page that provides information about the mission and vision of the organization?',
 'What is the URL of the page that provides information about the values and principles of the organization?',
 'What is the URL of the page that provides information about the leadership team of the organization?',
 'What is the URL of the page that provides information about the board of directors of the organization?',
 'What is the URL of the page that provides information about the staff of the organization?',
 'What is the URL of the page that provides information about the volunteers of the organization?',
 'What is the URL of the page that provides information about the donors of the organization?',
 'What is the URL of the page that provides information about the partners and sponsors of the organization?',
 'What is the URL of the page that provides information about the awards and recognition received by the organization?',
 'What is the URL of the page that provides information about the publications and resources offered by the organization?']

['What is the name of the organization that delivers health services to more than 4.4 million people living in Alberta?',
 'What is the purpose of Alberta Health Services (AHS)?',
 'How many direct AHS employees are there, excluding Covenant Health and other contracted service providers?',
 'How many staff work in AHS’ wholly-owned subsidiaries such as Alberta Precision Laboratories, Carewest and CapitalCare Group?',
 'What is the number of volunteers that support AHS?',
 'How many physicians practice in Alberta, approximately how many of them are members of the AHS medical staff?',
 'Where do students from Alberta’s universities and colleges receive clinical education?',
 'Where do students from universities and colleges outside of Alberta receive clinical education?',
 'What is the purpose of clinical education in AHS facilities and community locations?',
 'How many people does AHS serve in total?',
 'Which provinces does AHS serve, besides Alberta?',
 'What are the main responsibilities of AHS?',
 'What is the role of Covenant Health and other contracted service providers in relation to AHS?',
 'How many people does AHS serve in Saskatchewan, B.C. and the Northwest Territories?',
 'What is the purpose of excluding Covenant Health and other contracted service providers from the direct AHS employee count?',
 'What are the main goals of AHS?',
 'How does AHS work to achieve its goals?',
 'What is the role of volunteers in achieving AHS’ goals?',
 'What is the role of physicians in achieving AHS’ goals?',
 'What is the role of students from universities and colleges in achieving AHS’ goals?',
 'How does AHS work to ensure that its services are accessible to all residents of Alberta, regardless of their location or socioeconomic status?',
 'What is the role of community locations in providing health services to residents of Alberta?',
 'What is the role of AHS’ wholly-owned subsidiaries in providing health services to residents of Alberta?',
 'How does AHS work to ensure that its services are integrated and coordinated with other healthcare providers in Alberta?',
 'What is the role of AHS in ensuring that patients receive high-quality care?',
 'What is the role of AHS in ensuring that patients have access to the latest medical technologies and treatments?',
 'How does AHS work to ensure that its services are safe and effective?',
 'What is the role of AHS in ensuring that patients receive appropriate care for their specific needs?',
 'What is the role of AHS in ensuring that patients have access to healthcare services that meet their cultural and linguistic needs?',
 'What is the purpose of an equity partnership in primary care networks for AHS?1. What is the name of the organization mentioned in the text?',
 'What is the purpose of AHS?',
 'What are the three geographically based health authorities that were brought together by AHS?',
 'When was ground ambulance service added to the responsibilities of AHS?',
 'When were the services moved from municipalities to AHS?',
 'What is the vision of AHS?',
 'What is the shared purpose of AHS?',
 'What are the common goals of AHS?',
 'What is the People Strategy of AHS?',
 'What is the link to the People Strategy of AHS?',
 'What are the Employer Awards mentioned in the text?',
 'What is Canada’s Best Diversity Employers?',
 'What is Alberta’s Top 75 Employers?',
 'What is the link to the Employer Awards of AHS?',
 'What is the infographic mentioned in the text?',
 'What is the purpose of the infographic?',
 'What are the benefits of working at AHS according to the infographic?',
 'What is the link to the infographic of AHS?',
 'What is the mission statement of AHS?',
 'What is the role of leadership in achieving the vision and purpose of AHS?',
 'What are the resources needed by AHS to work together effectively?',
 'What is the link to the Leadership Strategy of AHS?',
 'What are the benefits of working at AHS according to the Leadership Strategy?',
 'What is the role of leadership in developing excellent leadership at AHS?',
 'What are the values that AHS respects, values and supports its people?',
 'What is the link to the Values Strategy of AHS?',
 'What are the benefits of working at AHS according to the Values Strategy?',
 'What is the role of leadership in creating a culture in which all feel safe, healthy and valued?',
 'What are the benefits of working at AHS according to the Culture Strategy?',
 'What is the link to the Culture Strategy of AHS?1. What is the name of the organization?',
 'What is the purpose of the infographic?',
 'Who is Dr. John Cowell and what is his role in AHS?',
 'Where can I find the Health Care Action Plan report?',
 'What are the bylaws and policies of AHS?',
 'What is the vision, mission, and strategic direction of AHS?',
 'What publications does AHS have available?',
 'What is the health plan and business plan of AHS?',
 'Where can I find contact information for AHS?',
 'How many pages are in the Health Care Action Plan report?',
 'Who wrote the Health Care Action Plan report?',
 'When was the Health Care Action Plan report released?',
 "What is the purpose of the Bylaws & Policies page on AHS's website?",
 'What are some examples of the policies listed on the Bylaws & Policies page?',
 'How can I find out more information about the Vision, Mission & Strategic Direction of AHS?',
 "What is the purpose of the Publications & Transparency page on AHS's website?",
 'What types of publications are listed on the Publications & Transparency page?',
 'How can I find out more information about the Health Plan & Business Plan of AHS?',
 "What is the purpose of the Contact Us page on AHS's website?",
 'What information is included on the Contact Us page?',
 'What is the purpose of the infographic?',
 'How many sections are in the infographic?',
 'What information is included in each section of the infographic?',
 'Who created the infographic?',
 'When was the infographic created?',
 'What is the purpose of the Health Care Action Plan report?',
 'What are some key points from the Health Care Action Plan report?',
 'How does AHS plan to address the challenges outlined in the Health Care Action Plan report?',
 'What is the role of Dr. John Cowell in addressing these challenges?',
 'What information is included in this subheading?1. What is the purpose of Alberta Health Services valuing patient feedback?',
 'How will patient feedback help improve the quality of healthcare in Alberta?',
 'What are the different ways patients can provide feedback to Alberta Health Services?',
 'What is the electronic feedback form for patient feedback?',
 'What is the mail or phone feedback option for patient feedback?',
 'What is the purpose of the Alberta Health Care Insurance Plan?',
 'What is the Alberta Health Care Card?',
 'Where can patients find information about the Alberta Health Care Insurance Plan and Alberta Health Care Card?',
 'What are the emergency phone numbers for patients in Alberta?',
 'Where can patients find information about health advisory councils in Alberta?',
 'How can patients send a complaint to Environmental Public Health in Alberta?',
 'What is the Health Inspector in Alberta?',
 'Where can patients find information about parking at healthcare facilities in Alberta?',
 'What is Patient Concerns & Feedback in Alberta Health Services?',
 'How can patients provide feedback about services they have received from Alberta Health Services?',
 'What is the purpose of Patient Well Wishes in Alberta Health Services?',
 'How can patients email a hospitalized patient in Alberta?',
 'Where can patients find information about patient well wishes in Alberta Health Services?',
 'What are the different types of feedback that patients can provide to Alberta Health Services?',
 'How does Alberta Health Services use patient feedback to improve healthcare services?',
 'What is the role of patient feedback in shaping healthcare policies in Alberta?',
 'How does Alberta Health Services ensure patient privacy and confidentiality when collecting feedback?',
 'What are the benefits of providing feedback to Alberta Health Services?',
 'How can patients ensure that their feedback is taken seriously by Alberta Health Services?',
 'What are the potential consequences of not providing feedback to Alberta Health Services?',
 'How does Alberta Health Services measure the success of its patient feedback initiatives?',
 'What are some examples of successful patient feedback initiatives in Alberta Health Services?',
 'How can patients stay informed about patient feedback initiatives in Alberta Health Services?',
 'What is the role of healthcare providers in collecting and using patient feedback in Alberta?',
 "Where can someone find general inquiries related to children's health services?1. What is the name of the location mentioned in the text?",
 'Which floor of the North Tower is mentioned in the text?',
 'What is the address of the location mentioned in the text?',
 'What is the city mentioned in the text?',
 'What is the province mentioned in the text?',
 'What is the postal code mentioned in the text?',
 'What is the phone number mentioned in the text?',
 'What is the URL of the page that mentions the location?',
 'What is the URL of the page where you can report a problem?',
 'What is the URL of the page that goes to the top of the website?',
 'What is the name of the organization mentioned in the text?',
 'What is the purpose of the website feedback form mentioned in the text?',
 'What is the URL of the page where you can find more information about the organization?',
 'What is the URL of the page that provides information about the services offered by the organization?',
 'What is the URL of the page that provides information about the programs and initiatives offered by the organization?',
 'What is the URL of the page that provides information about the history of the organization?',
 'What is the URL of the page that provides information about the mission and vision of the organization?',
 'What is the URL of the page that provides information about the values and principles of the organization?',
 'What is the URL of the page that provides information about the leadership team of the organization?',
 'What is the URL of the page that provides information about the board of directors of the organization?',
 'What is the URL of the page that provides information about the staff of the organization?',
 'What is the URL of the page that provides information about the volunteers of the organization?',
 'What is the URL of the page that provides information about the donors of the organization?',
 'What is the URL of the page that provides information about the partners and sponsors of the organization?',
 'What is the URL of the page that provides information about the awards and recognition received by the organization?',
 'What is the URL of the page that provides information about the publications and resources offered by the organization?']

In [ ]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader,TextLoader

def file_processing_ans(file_path):

    # Load data from PDF
    loader=DirectoryLoader("/content/data",glob="*.txt",loader_cls=TextLoader)
    data=loader.load()

    question_gen = ''

    for page in data:
        question_gen += page.page_content

    splitter_ques_gen = RecursiveCharacterTextSplitter(
        chunk_size = 1200,
        chunk_overlap = 100
    )

    chunks_ques_gen = splitter_ques_gen.split_text(question_gen)

    document_ques_gen = [Document(page_content=t) for t in chunks_ques_gen]

    splitter_ans_gen = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 50
    )


    document_answer_gen = splitter_ans_gen.split_documents(
        document_ques_gen
    )

    return document_ques_gen, document_answer_gen

In [ ]:
document_ques_gen, document_answer_gen=file_processing_ans("/content/data")

In [ ]:
document_answer_gen

[Document(page_content='Page No.1: ABOUT AHS\n\nABOUT AHS\n\nWHO WE ARE\n\nAlberta Health Services (AHS) is proud to be part of Canada’s first and largest provincewide, integrated health system, responsible for delivering health services to more than 4.4 million people living in Alberta, as well as to some residents of Saskatchewan, B.C. and the Northwest Territories.\n\nAHS has more than 112,300 direct AHS employees (excluding Covenant Health and other contracted service providers) and more than 13,000 staff working in AHS’ wholly-owned subsidiaries such as Alberta Precision Laboratories, Carewest and CapitalCare Group. AHS is also supported by approximately 9,100 volunteers and more than 11,000 physicians practicing in Alberta, approximately 9,000 of whom are members of the AHS medical staff (including active, temporary and community appointments).'),
 Document(page_content='Students from Alberta’s universities and colleges, as well as from universities and colleges outside of Albert

In [ ]:
embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vector_store = FAISS.from_documents(document_answer_gen, embeddings)

llm_answer_gen = load_llm()

answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen,
                                            chain_type="stuff",
                                            retriever=vector_store.as_retriever())

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import random

def get_csv (file_path):
    # answer_generation_chain, ques_list = llm_pipeline(file_path)
    base_folder = '/content/data'
    if not os.path.isdir(base_folder):
        os.mkdir(base_folder)
    output_file = base_folder+"QA.csv"
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Question", "Answer"])  # Writing the header row

        for question in random.sample(ques_list, 25):
            print("Question: ", question)
            answer = answer_generation_chain.run(question)
            print("Answer: ", answer)
            print("--------------------------------------------------\n\n")

            # Save answer to CSV file
            csv_writer.writerow([question, answer])
    return output_file

In [ ]:
output_file = get_csv("/content/data")

Question:  What are the different types of feedback that patients can provide to Alberta Health Services?
Answer:   Patients can provide feedback to Alberta Health Services through various channels such as electronic feedback form, mail or phone feedback, Health Advisory Councils, sending a complaint to Environmental Public Health, Health Inspector, and more. They can also thank the people who care, send personal well wishes to hospitalized patients, and provide general inquiries about health care services. Additionally, they can share their safety expectations, report a problem with the website, and go to the top of the page.
--------------------------------------------------


Question:  How will patient feedback help improve the quality of healthcare in Alberta?
Answer:   Patient feedback is valuable to Alberta Health Services as it helps them continuously improve the quality of healthcare in Alberta. By collecting and analyzing patient feedback, they can identify areas where improv

In [ ]:
output_file

'/content/dataQA.csv'

In [ ]:
answer_generation_chain("Generate a question from this text about a reference link to go to:To accomplish this, AHS has set a clear vision, with a shared purpose and common goals. AHS is committed to developing excellent leadership that will respect, value and support our people and provide the resources needed to work together effectively.Our People Strategy [https://www.albertahealthservices.ca/about/Page13140.aspx]")

{'query': 'Generate a question from this text about a reference link to go to:To accomplish this, AHS has set a clear vision, with a shared purpose and common goals. AHS is committed to developing excellent leadership that will respect, value and support our people and provide the resources needed to work together effectively.Our People Strategy [https://www.albertahealthservices.ca/about/Page13140.aspx]',
 'result': " What is the link to AHS's People Strategy?"}